In [55]:
import os

appsdir = '\\\\pbsstation\\pbsicmi\\Applications Engineering\\(4) Backup Bioreactor Databases'
cfgdir = '\\\\pbsstation\\pbsicmi\\Documents\\Manufacturing\\Mfg Released\\IM\\IM00226\\%s\\USB\\Install\\PBS Biotech Desktop\\1 Install RIO'

def getapp(sn):
    d1 = os.path.join(appsdir, sn)
    d2 = None
    ddate = 0
    if not os.path.exists(d1):
        raise FileNotFoundError(d1)
    for d in os.listdir(d1):
        if os.path.isdir(os.path.join(d1, d)) and d.startswith("factory"):
            factory, date = d.split()
            date = int(date)
            if date > ddate:
                ddate = date
                d2 = d
    if not d2:
        raise FileNotFoundError("Failed to find subdir for '%s'"%sn)
    return os.path.join(d1, d2)

def getapplv(sn):
    d = getapp(sn)
    return os.path.join(d, "LabVIEW_Data")
        
def get_lvdir(model, im226="IM00226 E"):
    return os.path.join(cfgdir%im226, model, "config\\LabVIEW data")

def lcmp(fn, ll1, ll2, doskip=lambda a,b,c: False):
    i = 0
    for l1, l2 in zip(ll1, ll2):
        if l1 != l2:
            try:
                s1 = l1.decode()
                s2 = l2.decode()
            except UnicodeDecodeError:
                s1 = repr(l1)
                s2 = repr(l2)
            if doskip(fn, s1, s2):
                continue
            print("'%s' != '%s'" % (s1, s2))
            i += 1
    return i

def fcmp(fn, f1, f2, doskip=lambda a,b,c: False):
    with open(f1, 'rb') as f:
        lines1 = f.read().splitlines()
    with open(f2, 'rb') as f:
        lines2 = f.read().splitlines()
    return lcmp(fn, lines1, lines2, doskip)

def skipline(fn, l1, l2):
    if fn == "Email Alerts Settings.xml":
        if l1.startswith("Subject") and l2.startswith("Subject"):
            return True
        if l1.startswith("Return Address") and l2.startswith("Return Address"):
            return True
    return False

skipfiles = set()
def isskip(s):
    return s in skipfiles

In [56]:
sns = [
    '000318N0404',
    '000318N0405',
    '000318N2201',
    '000318Q1701',
]

skipfiles = {
    'Email Password.cfgx',
}

reflv = get_lvdir("Mag 03", "IM00226 E")
mismatch = 0
for sn in sns:
    snlv = getapplv(sn)
    sn_files = os.listdir(snlv)
    for file in sn_files:
        if isskip(file):
            continue
        sfp = os.path.join(snlv, file)
        rfp = os.path.join(reflv, file)
        mismatch += fcmp(file, sfp, rfp, skipline)
        
print("%d mismatching items"%mismatch)

0 mismatching items
